In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def inception_module(x, f1x1, f3x3_reduce, f3x3, f5x5_reduce, f5x5, pool_proj):
    conv_1x1 = layers.Conv2D(f1x1, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = layers.Conv2D(f3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = layers.Conv2D(f3x3, (3, 3), padding='same', activation='relu')(conv_3x3)
    conv_5x5 = layers.Conv2D(f5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = layers.Conv2D(f5x5, (5, 5), padding='same', activation='relu')(conv_5x5)
    pool_proj = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = layers.Conv2D(pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)
    output = layers.Concatenate(axis=-1)([conv_1x1, conv_3x3, conv_5x5, pool_proj])
    return output

In [3]:
def build_googlenet(input_shape=(224, 224, 3), num_classes=1000):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    x = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = layers.Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    x = inception_module(x, f1x1=64, f3x3_reduce=96, f3x3=128, f5x5_reduce=16, f5x5=32, pool_proj=32)
    x = inception_module(x, f1x1=128, f3x3_reduce=128, f3x3=192, f5x5_reduce=32, f5x5=96, pool_proj=64)
    
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    x = inception_module(x, f1x1=192, f3x3_reduce=96, f3x3=208, f5x5_reduce=16, f5x5=48, pool_proj=64)
    
    aux1 = layers.AveragePooling2D((5, 5), strides=3)(x)
    aux1 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(aux1)
    aux1 = layers.Flatten()(aux1)
    aux1 = layers.Dense(1024, activation='relu')(aux1)
    aux1 = layers.Dense(num_classes, activation='softmax')(aux1)
    
    x = inception_module(x, f1x1=160, f3x3_reduce=112, f3x3=224, f5x5_reduce=24, f5x5=64, pool_proj=64)
    x = inception_module(x, f1x1=128, f3x3_reduce=128, f3x3=256, f5x5_reduce=24, f5x5=64, pool_proj=64)
    x = inception_module(x, f1x1=112, f3x3_reduce=144, f3x3=288, f5x5_reduce=32, f5x5=64, pool_proj=64)
    
    aux2 = layers.AveragePooling2D((5, 5), strides=3)(x)
    aux2 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(aux2)
    aux2 = layers.Flatten()(aux2)
    aux2 = layers.Dense(1024, activation='relu')(aux2)
    aux2 = layers.Dense(num_classes, activation='softmax')(aux2)
    
    x = inception_module(x, f1x1=256, f3x3_reduce=160, f3x3=320, f5x5_reduce=32, f5x5=128, pool_proj=128)
    
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    x = inception_module(x, f1x1=256, f3x3_reduce=160, f3x3=320, f5x5_reduce=32, f5x5=128, pool_proj=128)
    x = inception_module(x, f1x1=384, f3x3_reduce=192, f3x3=384, f5x5_reduce=48, f5x5=128, pool_proj=128)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, [outputs, aux1, aux2])
    
    return model

In [ ]:
googlenet_scratch_model = build_googlenet()
googlenet_scratch_model.summary()